# Демонстрация использования класса `OvervoltageDetector`

Этот ноутбук показывает, как использовать класс `OvervoltageDetector` из `analysis.overvoltage_detector` для анализа осциллограмм на предмет перенапряжений во время однофазных замыканий на землю (ОЗЗ).

Ключевые этапы:
- Загрузка и подготовка данных осциллограммы (`Oscillogram`).
- Нормализация данных (`OscillogramNormalizer`).
- Разделение данных по шинам (`OscillogramToCsvConverter`).
- Поиск зон ОЗЗ и расчет максимального коэффициента перенапряжения в этих зонах.

**Важно**: Этот ноутбук использует образцы файлов из директории `tests/sample_data/`. Для демонстрации файлы копируются во временную рабочую директорию (`temp_overvoltage_notebook_data/...`). Убедитесь, что тестовые данные были сгенерированы запуском скрипта `tests/test_data_setup.py`.

In [ ]:
import os
import shutil
import pandas as pd
import numpy as np
import json 
import csv 
import sys

# --- Пути к данным для тестирования ---
module_path = os.path.abspath(os.path.join('..')) 
if module_path not in sys.path:
    sys.path.append(module_path)
    
from core.oscillogram import Oscillogram
from normalization.normalization import OscillogramNormalizer
from raw_to_csv.raw_to_csv import OscillogramToCsvConverter
from analysis.overvoltage_detector import OvervoltageDetector

base_sample_data_dir = os.path.join(module_path, "tests", "sample_data")
comtrade_samples_for_ov_path = os.path.join(base_sample_data_dir, "comtrade_files", "for_spef_overvoltage_1")
config_samples_path = os.path.join(base_sample_data_dir, "config_files")

# --- Создание временной рабочей директории для этого ноутбука ---
notebook_base_temp_dir_ov = "temp_overvoltage_notebook_data"
notebook_source_dir_ov = os.path.join(notebook_base_temp_dir_ov, "source_oscillograms_ov")
notebook_output_dir_ov = os.path.join(notebook_base_temp_dir_ov, "overvoltage_analysis_output")
notebook_config_dir_ov = os.path.join(notebook_base_temp_dir_ov, "config_files_ov")

if os.path.exists(notebook_base_temp_dir_ov):
    shutil.rmtree(notebook_base_temp_dir_ov)
os.makedirs(notebook_source_dir_ov)
os.makedirs(notebook_output_dir_ov)
os.makedirs(notebook_config_dir_ov)

# --- Копирование необходимых файлов для демонстрации ---
all_files_copied_ov = True
# 1. Осциллограммы (spef_present.cfg, no_spef.cfg, spef_symmetrical_phases.cfg, etc.)
ov_cfg_files_to_copy = ["spef_present.cfg", "no_spef.cfg", "spef_symmetrical_phases.cfg", "spef_low_ov.cfg"]
for cfg_base_name in ov_cfg_files_to_copy:
    # Копируем CFG
    src_cfg_path = os.path.join(comtrade_samples_for_ov_path, cfg_base_name)
    if os.path.exists(src_cfg_path):
        shutil.copy(src_cfg_path, notebook_source_dir_ov)
    else:
        print(f"ОШИБКА: Исходный файл CFG не найден: {src_cfg_path}")
        all_files_copied_ov = False
        continue
    # Копируем DAT
    dat_file_name = cfg_base_name[:-4] + ".dat"
    src_dat_path = os.path.join(comtrade_samples_for_ov_path, dat_file_name)
    if os.path.exists(src_dat_path):
        shutil.copy(src_dat_path, notebook_source_dir_ov)
    else:
        print(f"ОШИБКА: Исходный файл DAT не найден: {src_dat_path}")
    
# 2. Конфигурационный JSON для OvervoltageDetector
src_ov_config_json = os.path.join(config_samples_path, "overvoltage_detector_config.json")
dest_ov_config_json_ov = os.path.join(notebook_config_dir_ov, "overvoltage_detector_config.json")
if os.path.exists(src_ov_config_json): shutil.copy(src_ov_config_json, dest_ov_config_json_ov)
else: print(f"ОШИБКА: overvoltage_detector_config.json не найден в {config_samples_path}."); all_files_copied_ov = False

# 3. Файл коэффициентов нормализации (используемый OvervoltageDetector через Normalizer)
src_norm_coeffs_ov_path = os.path.join(config_samples_path, "norm_coeffs_for_ov_test.csv")
dest_norm_coeffs_ov_path = os.path.join(notebook_config_dir_ov, "norm_coeffs_for_ov_test.csv")
if os.path.exists(src_norm_coeffs_ov_path): shutil.copy(src_norm_coeffs_ov_path, dest_norm_coeffs_ov_path)
else: print(f"ОШИБКА: norm_coeffs_for_ov_test.csv не найден в {config_samples_path}."); all_files_copied_ov = False
        
# 4. JSON словари для имен каналов (для bus_splitter)
src_analog_json_ov = os.path.join(config_samples_path, "dict_analog_names_sample.json") # Reusing general sample
dest_analog_json_ov = os.path.join(notebook_config_dir_ov, "current_analog_names.json")
if os.path.exists(src_analog_json_ov): shutil.copy(src_analog_json_ov, dest_analog_json_ov)
else: print(f"ОШИБКА: dict_analog_names_sample.json не найден в {config_samples_path}."); all_files_copied_ov = False
        
src_discrete_json_ov = os.path.join(config_samples_path, "dict_discrete_names_sample.json")
dest_discrete_json_ov = os.path.join(notebook_config_dir_ov, "current_discrete_names.json")
if os.path.exists(src_discrete_json_ov): shutil.copy(src_discrete_json_ov, dest_discrete_json_ov)
else: print(f"ОШИБКА: dict_discrete_names_sample.json не найден в {config_samples_path}."); all_files_copied_ov = False

if not all_files_copied_ov:
    print("Не все критичные файлы были скопированы. Демонстрация может быть неполной.")
else:
    print(f"Демонстрационные файлы и конфигурации скопированы во временные директории в: {notebook_base_temp_dir_ov}")
    print("Содержимое source_oscillograms_ov:")
    for item in os.listdir(notebook_source_dir_ov): print(f"- {item}")

## 1. Инициализация компонентов

In [ ]:
ov_config = {
    # Эта конфигурация будет загружена из файла
}
if os.path.exists(dest_ov_config_json_ov):
    with open(dest_ov_config_json_ov, 'r', encoding='utf-8') as f_cfg:
        loaded_ov_config = json.load(f_cfg)
else:
    print(f"ОШИБКА: Файл конфигурации {dest_ov_config_json_ov} не найден. Используется заглушка.")
    loaded_ov_config = { 'VALID_NOMINAL_VOLTAGES': [6000/np.sqrt(3)], 'SPEF_THRESHOLD_U0':0.05, 'SPEF_MIN_DURATION_PERIODS':1 }

normalizer_ov = OscillogramNormalizer(norm_coef_file_path=dest_norm_coeffs_ov_path, is_print_message=True)

bus_splitter_ov = OscillogramToCsvConverter(
    normalizer=normalizer_ov, 
    raw_path=notebook_source_dir_ov, 
    csv_path=notebook_output_dir_ov, # Используется для некоторых методов bus_splitter, если они вызываются напрямую
    dict_analog_names_path=dest_analog_json_ov,
    dict_discrete_names_path=dest_discrete_json_ov,
    is_print_message=False, 
    show_progress_bars=False 
)

norm_coeffs_df_ov = None
if os.path.exists(dest_norm_coeffs_ov_path):
    norm_coeffs_df_ov = pd.read_csv(dest_norm_coeffs_ov_path)
else:
    print(f"ОШИБКА: Файл с коэффициентами нормализации {dest_norm_coeffs_ov_path} не найден. Detector может работать некорректно.")
    # Создаем пустой DataFrame, чтобы избежать ошибки при инициализации Detector
    norm_coeffs_df_ov = pd.DataFrame(columns=['name', 'norm', '1Ub_base']) 

detector = OvervoltageDetector(
    config=loaded_ov_config,
    normalizer=normalizer_ov,
    bus_splitter=bus_splitter_ov,
    norm_coef_df=norm_coeffs_df_ov, 
    show_progress_bars=True
)
print("\nOvervoltageDetector и зависимости инициализированы.")

## 2. Анализ директории на перенапряжения при ОЗЗ (`analyze_directory`)

In [ ]:
output_ov_report_csv = os.path.join(notebook_output_dir_ov, "overvoltage_report.csv")
error_log_ov_path = os.path.join(notebook_output_dir_ov, "overvoltage_errors.log")

print(f"\nЗапуск анализа директории: {notebook_source_dir_ov}")
detector.analyze_directory(
    osc_folder_path=notebook_source_dir_ov,
    output_csv_path=output_ov_report_csv,
    log_path=error_log_ov_path
)

if os.path.exists(output_ov_report_csv):
    print(f"\nСодержимое отчета о перенапряжениях ({output_ov_report_csv}):")
    try: 
        from IPython.display import display
        report_df = pd.read_csv(output_ov_report_csv)
        display(report_df)
    except ImportError:
        print(pd.read_csv(output_ov_report_csv))
    except Exception as e:
        print(f"Ошибка чтения CSV отчета: {e}")
else:
    print(f"Отчет о перенапряжениях не был создан.")

if os.path.exists(error_log_ov_path) and os.path.getsize(error_log_ov_path) > 0:
    print(f"\nСодержимое лога ошибок ({error_log_ov_path}):")
    with open(error_log_ov_path, "r", encoding='utf-8') as f: print(f.read())
else:
    print(f"\nЛог ошибок ({error_log_ov_path}) пуст или не создан.")

## 3. Демонстрация `analyze_oscillogram` (для одного файла)

In [ ]:
try:
    osc_example_ov_path = os.path.join(notebook_source_dir_ov, "spef_present.cfg") # Используем файл с ОЗЗ из набора
    osc_obj_ov = Oscillogram(osc_example_ov_path)
    print(f"\nАнализ одного файла: {osc_obj_ov.file_hash}")
    
    # detector.verbose = True # Already set in config for this demo
    result_single = detector.analyze_oscillogram(osc_obj_ov)

    if result_single:
        print("Результат анализа для одного файла:")
        print(result_single)
    else:
        print("Для данного файла значимых перенапряжений при ОЗЗ не найдено или файл не прошел проверки.")
except Exception as e:
    print(f"Ошибка при анализе одного файла: {e}")

## 4. Демонстрация `copy_spef_oscillograms` (статический метод)

In [ ]:
copied_ov_dir = os.path.join(notebook_output_dir_ov, "copied_overvoltage_oscillograms")

if os.path.exists(output_ov_report_csv): 
    print(f"\nКопирование осциллограмм из отчета {output_ov_report_csv} в {copied_ov_dir}...")
    OvervoltageDetector.copy_spef_oscillograms(
        report_csv_path=output_ov_report_csv,
        source_osc_folder_path=notebook_source_dir_ov, 
        destination_folder_path=copied_ov_dir,
        is_print_message=True # Статический метод, is_print_message или verbose_logging_method
    )
    
    if os.path.exists(copied_ov_dir):
        print("\nСодержимое директории со скопированными осциллограммами:")
        for item in os.listdir(copied_ov_dir): print(f"- {item}")
else:
    print(f"Отчет {output_ov_report_csv} не найден, копирование не будет выполнено.")

In [ ]:
# Очистка временной директории
try:
    if os.path.exists(notebook_base_temp_dir_ov):
        shutil.rmtree(notebook_base_temp_dir_ov)
        print(f"\nВременная директория {notebook_base_temp_dir_ov} удалена.")
    else:
        print(f"\nВременная директория {notebook_base_temp_dir_ov} не найдена для удаления.")
except Exception as e:
    print(f"Ошибка при удалении временной директории {notebook_base_temp_dir_ov}: {e}")